In [1]:
from config import aws_endpoint, aws_master_username, aws_master_password, database
import mysql.connector
from datetime import datetime
import pandas as pd

In [80]:
class SQL:
    def __init__(self, host, user, password, database):
        self.host = host
        self.user = user
        self.password = password
        self.database = database
        self.my_database = mysql.connector.connect(
                            host=self.host,
                            user=self.user,
                            password=self.password,
                            database=self.database
                        )
        self.my_cursor = self.my_database.cursor()

    def query_data(self, query:str):
        """Provides the ability to query data from the database

        Args:
            query (str): SQL query

        Raises:
            Exception: Must pass str for query
            e: SQL syntax based error

        Returns:
            df (pd.DataFrame): Dataframe
        """
        if not isinstance(query,str): 
            raise Exception("Must pass str for query")
        
        try:
            self.my_cursor.execute(query)
            columns = self.my_cursor.description
            data = [{columns[index][0]:column for index, column in enumerate(value)} for value in self.my_cursor.fetchall()]
            df = pd.DataFrame(data)
            return df
        except Exception as e:
            raise e
        
    def insert_data(self, query:str):
        """Calls the initate function then passes the query to insert data

        Args:
            query (str): SQL query function

        Exception: Must pass str type for query variable 
        """
        if not isinstance(query,str): 
            raise Exception("Must pass str for query")
                    
        try:
            pass
            self.my_cursor.execute(query)
            self.my_database.commit()
        except Exception as e:
            raise e


class Lifts(SQL): 
    def new_lift(self, user_id:int, lift:int, set_number:int, reps:int, weight:int, lift_date:str):
        """Provide ability to import into the lift database

        Args:
            set_number (int): Set Number
            reps (int): Reps performed
            weight (int): Weight lifted for each rep in the set
            lift_date (str): Date performed

        Exception: Must pass int for user_id
        Exception: Must provide integar type for set_number, reps, weight, lift_date
        """
        if not isinstance(user_id, int): 
            raise Exception("Must pass int for user_id")
        if not all([isinstance(lift,int),isinstance(set_number,int),isinstance(reps,int),isinstance(weight,int)]):
            raise Exception("Please provide integar type for set_number, reps, weight, lift_date")

        query = f"""INSERT INTO WEIGHTLIFTING.LIFTS (USER_ID,LIFT,SET_NUMBER,REPS,WEIGHT,LIFT_DATE) 
                    VALUES ({user_id}, {lift},{set_number},{reps},{weight},'{lift_date}');"""

        SQL.insert_data(self,query)

    
class Weight(SQL):
    def new_weight(self, user_id:int, weight:float, weight_date:str):
        try: 
            weight = round(float(weight), 2)
        except Exception:
            raise Exception("Must provide float type for weight")

        if not isinstance(weight_date,str):
            raise Exception("Must provide str type for weight_date")

        query = f"""INSERT INTO WEIGHTLIFTING.BODY_WEIGHT (USER_ID, WEIGHT, WEIGHT_DATE) 
                    VALUES ({user_id}, {weight},'{weight_date}');"""
        
        SQL.insert_data(self,query)

class User(SQL):
    def new_user(self, first_name:str, last_name:str, email:str, user_password:str):
        if not all([isinstance(first_name, str), isinstance(last_name, str), isinstance(email, str), isinstance(user_password, str)]):
            raise Exception("Please pass str for each variable")

        # Dupe Email check
        if len(W.query_data(f"""SELECT * FROM WEIGHTLIFTING.USERS U WHERE LOWER(U.EMAIL) LIKE LOWER('{email}')""")) > 0:
            raise Exception("Email address already in use. Please choose another.")        

        query = f"""INSERT INTO WEIGHTLIFTING.USERS (FIRST_NAME,LAST_NAME,EMAIL,USER_PASSWORD) 
                    VALUES ('{first_name.strip()}', '{last_name.strip()}', '{email}', '{user_password}');"""

        SQL.insert_data(self,query)

In [69]:
' hel '.strip()

'hel'

In [76]:
# W = Weight(aws_endpoint,aws_master_username,aws_master_password,database)
# # W.new_weight(5, 100.5456456, '1999-01-01')
# test = 'john.smith@gmail.com'
# len(W.query_data(f"""SELECT *
#                     FROM WEIGHTLIFTING.USERS U
#                     WHERE LOWER(U.EMAIL) LIKE LOWER('{test}')"""))

1

In [52]:
lift = Lifts(aws_endpoint,aws_master_username,aws_master_password,database)
lift.new_lift(1,0,0,0,0,'2000-01-1')

In [82]:
u = User(aws_endpoint,aws_master_username,aws_master_password,database)
u.new_user('John', 'Smith', 'john.smith2@gmail.com', 'a_password_ayo')

In [36]:
S = SQL(aws_endpoint,aws_master_username,aws_master_password,database)
x = S.query_data("select * from weightlifting.lifts")

ProgrammingError: 1049 (42000): Unknown database 'weightlifting'

In [44]:
lift = Lifts(aws_endpoint,aws_master_username,aws_master_password,database)
lift.new_lift(0,0,0,0,'2000-01-1')

ProgrammingError: 1054 (42S22): Unknown column 'BODY_WEIGHT' in 'field list'

In [15]:
# weight = Weight(aws_endpoint,aws_master_username,aws_master_password,database)
# weight.new_weight(165, '2015-01-01')
database

'WEIGHTLIFITNG'

In [69]:
lift = Lifts(aws_endpoint,aws_master_username,aws_master_password,database)
# lift.my_cursor.execute("""select * from WEIGHTLIFTING.LIFTS""")
# for x in lift.my_cursor.fetchall():
#     print(x)

In [24]:
sql = Lifts(aws_endpoint,aws_master_username,aws_master_password,database)
df = sql.query_data("""select * from WEIGHTLIFTING.USERS""")

In [25]:
df

,ENTITY_ID,FIRST_NAME,LAST_NAME,EMAIL,USER_PASSWORD,CREATED_DATE,DELETED
0,1,John,Smith,john.smith@gmail.com,a_password_ayo,2022-01-23 19:01:12,0


In [65]:
columns = lift.my_cursor.description
data = [{columns[index][0]:column for index, column in enumerate(value)} for value in lift.my_cursor.fetchall()]
df = pd.DataFrame(result)


In [67]:
pd.DataFrame(result)

,ENTITY_ID,LIFT,SET_NUMBER,REPS,WEIGHT,LIFT_DATE
0,1,1,2,10,135,2022-01-22
1,2,2,3,8,155,NaT
2,3,5,7,34,234565,NaT
3,4,5,7,34,234565,2002-01-23
4,6,5,7,34,234565,2002-01-23
5,7,0,0,0,0,2000-01-01
6,8,0,0,0,0,2000-01-01


In [9]:
from config import aws_endpoint, aws_master_username, aws_master_password, database


my_database = mysql.connector.connect(
                            host=aws_endpoint,
                            user=aws_master_username,
                            password=aws_master_password,
                            database=database
                        )
my_cursor = my_database.cursor()

In [13]:
# my_cursor.execute("SELECT * FROM WEIGHTLIFTING.LIFT_EXERCISE")
columns = my_cursor.description
data = [{columns[index][0]:column for index, column in enumerate(value)} for value in my_cursor.fetchall()]
df = pd.DataFrame(data)

In [15]:
my_cursor.execute("""INSERT INTO WEIGHTLIFTING.USERS (FIRST_NAME,LAST_NAME,EMAIL,USER_PASSWORD) 
VALUES ('John', 'Smith', 'john.smith@gmail.com', 'a_password_ayo');""")
my_database.commit()

In [37]:
s = SQL(aws_endpoint, aws_master_username, aws_master_password, database)

In [38]:
s.query_data("""SELECT *
FROM WEIGHTLIFTING.USERS""")

,ENTITY_ID,FIRST_NAME,LAST_NAME,EMAIL,USER_PASSWORD,CREATED_DATE,DELETED
0,1,John,Smith,john.smith@gmail.com,a_password_ayo,2022-01-23 19:01:12,0
